In [ ]:
import os

os.chdir("../")
%pwd

In [16]:
os.environ["MLFLOW_TRACKING_URI"]="your_mlflow_tracking_uri"
os.environ["MLFLOW_TRACKING_USERNAME"]="your_mlflow_tracking_username"
os.environ["MLFLOW_TRACKING_PASSWORD"]="your_mlflow_tracking_access_token"

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_dir: Path
    model_dir: Path
    all_params: dict
    metrics_file_name: str
    target_column: str
    mlflow_uri: str

In [25]:
from MLproject.constants import *
from MLproject.utils.common import read_yaml, create_directories, save_json

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
            
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.Elasticnet
        schema = self.schema.TARGET_COLUMN
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_dir=config.test_data_dir,
            model_dir=config.model_dir,
            metrics_file_name=config.metrics_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Harshvardhan2164/MLOPS-Project.mlflow"
        )
        
        return model_evaluation_config

In [26]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
import joblib

In [27]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_dir)
        model = joblib.load(self.config.model_dir)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri("your_mlflow_tracking_uri")
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            
            metrics_dir = os.path.dirname(self.config.metrics_file_name)
            os.makedirs(metrics_dir, exist_ok=True)
            save_json(path=Path(self.config.metrics_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2", r2)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [24]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-02-01 12:31:42,061: INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='cp1252'> loaded successfully]
[2025-02-01 12:31:42,061: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='cp1252'> loaded successfully]
[2025-02-01 12:31:42,069: INFO: common: yaml file: <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='cp1252'> loaded successfully]
[2025-02-01 12:31:42,069: INFO: common: Created directory at: artifacts]


C:\Users\Harshvardhan Sharma\AppData\Roaming\Python\Python310\site-packages\_distutils_hack\__init__.py:36: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticNetModel'.
2025/02/01 12:31:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.
